In [1]:
#import matplotlib.pyplot as plt
from keras import backend as K
from keras import Model
from keras.layers import Dense,Input,concatenate
from keras.utils import to_categorical
from keras.optimizers import SGD
from keras.callbacks import Callback
from  sklearn.utils import resample
import functools
import pandas
import tensorflow as tf
import numpy as np
from tensorflow.python.saved_model import builder as saved_model_builder
from tensorflow.python.saved_model import tag_constants, signature_constants, signature_def_utils_impl

/home/fuckme/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
df=pandas.read_csv('Logs/stats.csv')
print(len(df))

2596153


In [3]:
df["Opponent_Percent_Change"] = df["Opponent_Percent_Change"].shift(-1)
df["AI_Percent_Change"] = df["AI_Percent_Change"].shift(-1)
df.drop(len(df)-1,inplace=True)

In [4]:
df['target'] = df.apply (lambda row: row["Opponent_Percent_Change"]>0,axis=1)
df[["Opponent_Facing", "AI_Facing","target"]] *= 1
print(len(df[(df['target'] > 0)]),len(df))

8517 2596152


In [5]:
discount_factor=.99
current_reward=0
for i in reversed(df.index):
    df.at[i,"target"]=df.loc[i,"target"]+discount_factor*current_reward
    current_reward=df.loc[i,"target"]

In [6]:
df=df.drop(columns=['AI_Action', 'Opponent_Action', 'Buttons Pressed', 'Notes', 'Frame Process Time','Opponent_Percent_Change','AI_Percent_Change'])

In [8]:
sess = tf.Session()
K.set_session(sess)
K.set_learning_phase(0)  # all new operations will be in test mode from now on
model_version = "2"
seed = 7
np.random.seed(seed)

In [15]:
Y_train=df['target'].astype(float).values
X_train=df[df.columns.difference(['target', 'Buttons_Pressed_Converted'])].astype(float).values
action_train=df['Buttons_Pressed_Converted'].astype(int).values

In [19]:
# Input layer
inputs = Input(shape=(17,))
action = Input(shape=(1,))
# Fill in your own hidden layers
action_model = Dense(8, activation='relu')(action)
input_and_actions= concatenate([inputs,action_model])
x = Dense(128, activation='relu')(input_and_actions)
x = Dense(64, activation='relu')(x)
x = Dense(32, activation='relu')(x)
x = Dense(1, kernel_initializer='normal')(x)
model = Model(inputs=[inputs,action], outputs=x)
model.compile(optimizer="Adam", loss='mae')
history=model.fit([X_train,action_train], Y_train,batch_size=128, epochs=2,validation_split=.2,verbose=1)

Train on 2076921 samples, validate on 519231 samples
Epoch 1/2
2076921/2076921 [==============================] - 27s 13us/step - loss: 0.3340 - val_loss: 0.2761
Epoch 2/2
2076921/2076921 [==============================] - 28s 13us/step - loss: 0.2793 - val_loss: 0.2667


In [20]:
results=model.predict([X_train[0:5],action_train[0:5]])
print(np.hstack(results))
print(Y_train[0:5])

[0.04566531 0.04557688 0.04577363 0.04563114 0.04566564]
[0.02519006 0.0254445  0.02570152 0.02596113 0.02622336]


In [57]:
for i in range(1):
    X_test=np.tile(X_train[i],(149,1))
    action_test=np.array(range(149))
    result=model.predict([X_test,action_test])

In [63]:
def softmax(x):
    """Compute softmax values for each sets of scores in x."""
    e_x = np.exp(x - np.max(x))
    return e_x / e_x.sum(axis=0)


In [71]:
x=softmax(result)

np.random.choice(x,p=x)

[0.00670972]


ValueError: a must be 1-dimensional

In [22]:
json_string = model.to_json()
with open("model3.json", "w") as json_file:
    json_file.write(json_string)
model.save_weights('model3.h5')    

In [ ]:
np.argmax(model.predict([X_test,action_test]))

In [ ]:
import numpy as np
np.argmax(results)

In [ ]:
labels=["True","False"]
y=auto.input_nub

In [ ]:
classification_inputs

In [ ]:
values, indices = tf.nn.top_k(y, len(labels))
table =tf.contrib.lookup.index_to_string_table_from_tensor(tf.constant(labels))
prediction_classes = table.lookup(tf.to_int64(indices))

classification_inputs = tf.saved_model.utils.build_tensor_info(auto.input_nub)
classification_outputs_classes = tf.saved_model.utils.build_tensor_info(prediction_classes)
classification_outputs_scores = tf.saved_model.utils.build_tensor_info(values)

classification_signature = tf.saved_model.signature_def_utils.classification_signature_def(
    examples=classification_inputs,
    classes=prediction_classes,
    scores=classification_outputs_scores
)
prediction_signature = tf.saved_model.signature_def_utils.predict_signature_def({"inputs": x}, {"prediction":y})

In [ ]:
valid_prediction_signature = tf.saved_model.signature_def_utils.is_valid_signature(prediction_signature)
valid_classification_signature = tf.saved_model.signature_def_utils.is_valid_signature(classification_signature)

if(valid_prediction_signature == False):
    raise ValueError("Error: Prediction signature not valid!")

if(valid_classification_signature == False):
    raise ValueError("Error: Classification signature not valid!")

In [ ]:
builder = saved_model_builder.SavedModelBuilder('./'+model_version)
legacy_init_op = tf.group(tf.tables_initializer(), name='legacy_init_op')

# Add the meta_graph and the variables to the builder
builder.add_meta_graph_and_variables(
      sess, [tag_constants.SERVING],
      signature_def_map={
           'predict-smash':
               prediction_signature,
           signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY:
              classification_signature,
      },
      legacy_init_op=legacy_init_op)
# save the graph
builder.save()

In [ ]:
classfication_signature = tf.saved_model.signature_def_utils.classification_signature_def(examples=model.input,classes=model.output,scores=model.output)

In [ ]:
print(history.history.keys())

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# TODO: account for end of game
# Discount Factor Wait on this
for idx in reversed(df.index):
    print(idx, df.loc[idx, "AI_Percent_Change"])

In [ ]:
categorical_vars=["Opponent_Facing","AI_Facing","target"]
numerical_vars=["Frame","Opponent_x","Opponent_y","AI_x","AI_y",
                "Opponent_Action_Num","AI_Action_Num","AI_Action_Frame",
                "Opponent_Action_Frame","Opponent_Jumps_Left", "AI_Jumps_Left",
                "Opponent_Stock","AI_Stock",
                "Opponent_Percent","AI_Percent","Buttons_Pressed_Converted"]
auto=Automater(categorical_vars=categorical_vars,numerical_vars=numerical_vars,response_var="target")

In [39]:
df=pandas.read_csv('Logs/stats.csv')

In [42]:
for i in range(len(df.head(1).keys())):
    print("\"{}\":{},".format(df.head(1).keys()[i],df.head(1).values[0][i]))

"Frame":1,
"Opponent_x":-70.0,
"Opponent_y":7.0,
"AI_x":70.0,
"AI_y":7.0,
"Opponent_Facing":True,
"AI_Facing":False,
"Opponent_Action":Action.ENTRY,
"Opponent_Action_Num":322,
"AI_Action":Action.ENTRY,
"AI_Action_Num":322,
"Opponent_Action_Frame":-1,
"AI_Action_Frame":-1,
"Opponent_Jumps_Left":1,
"AI_Jumps_Left":1,
"Opponent_Stock":4,
"AI_Stock":4,
"Opponent_Percent":0,
"AI_Percent":0,
"Opponent_Percent_Change":0,
"AI_Percent_Change":0,
"Buttons Pressed":SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 1
SET MAIN 0.5 0.5
PRESS A
SET MA

In [ ]:
X, y = auto.fit_transform(df)